In [34]:
from ipywidgets import interact
from matplotlib import pyplot as plt
import time
from sequence.kernel.timeline import Timeline
from sequence.topology.node import QuantumRouter, BSMNode
from sequence.components.optical_channel import QuantumChannel, ClassicalChannel
from sequence.resource_management.rule_manager import Rule
from sequence.entanglement_management.generation import EntanglementGenerationA


# our rule condition requires RAW (unentangled) memories
def eg_rule_condition(memory_info, manager, args):
    if memory_info.state == "RAW":
        return [memory_info]
    else:
        return []

# define action to be taken when we meet our condition on router 1
def eg_rule_action1(memories_info, args):
    # define requirement of protocols on other node
    def eg_req_func(protocols, args):
        for protocol in protocols:
            if isinstance(protocol, EntanglementGenerationA):
                return protocol
            
    # create entanglement generation protocol with proper parameters
    memories = [info.memory for info in memories_info]
    memory = memories[0]
    protocol = EntanglementGenerationA(None, "EGA." + memory.name, "m1", "r2", memory)
    protocol.primary = True
    
    # return values for a rule are: the protocol created, the destination node,
    # the destination codition, and arguments for the condition.
    return [protocol, ["r2"], [eg_req_func], [None]]

# define action to be taken when we meet our condition on router 2
def eg_rule_action2(memories_info, args):
    memories = [info.memory for info in memories_info]
    memory = memories[0]
    protocol = EntanglementGenerationA(None, "EGA." + memory.name, "m1", "r1", memory)
    return [protocol, [None], [None], [None]]






def test(cc_delay, qc_atten,qc_dist_range):
    
    """
    
    cc_delay: delay on classical channels (ms)
    qc_atten: attenuation on quantum channels (db/m)
    
    """
    PS_PER_MS = 1e9
    M_PER_KM = 1e3
    
    # convert units for cc delay (to ps) and qc distance (to m)
    cc_delay *= PS_PER_MS
    
    
    # construct the simulation timeline; the constructor argument is the simulation time (in ps) fo one second
    tl = Timeline(1000 * PS_PER_MS)
    
    # first, construct the quantum routers
    # (with arguments for the node name, timeline, and number of quantum memories)
    r1 = QuantumRouter("r1", tl)
    r2 = QuantumRouter("r2", tl)
    # next, construct the BSM nodes
    # (with arguments for the node name, timeline, and the names of connected routers)
    m1 = BSMNode("m1", tl, ["r1", "r2"])
    r1.set_seed(0)
    r2.set_seed(1)
    m1.set_seed(2)
    
    for node in [r1, r2]:
        # update coherence time parameter for node memories (measured in seconds)
        node.memory_array.update_memory_params("coherence_time", 0.3)
    
    # create all-to-all classical connections
    nodes = [r1, r2, m1]
    for node1 in nodes:
        for node2 in nodes:
            if node1 == node2:
                continue
            # construct a classical communication channel
            # (with arguments for the channel name, timeline, length (in m), and delay (in ps))
            cc = ClassicalChannel("_".join(["cc", node1.name, node2.name]), tl, 1e3, delay=cc_delay)
            cc.set_ends(node1, node2.name)
    data = []
    for distance in range(1):
        
        # create linear quantum network between routers and middle node
        # for this, we create quantum channels
        # (with arguments for the channel name, timeline, attenuation (in dB/km), and distance (in m))
        #qc_dist = distance*M_PER_KM
        qc_dist = qc_dist_range*M_PER_KM
        print(qc_dist)
        qc1 = QuantumChannel("qc_r1_m1", tl, qc_atten, qc_dist)
        qc1.set_ends(r1, m1.name)
        qc2 = QuantumChannel("qc_r2_m1", tl, qc_atten, qc_dist)
        qc2.set_ends(r2, m1.name)

        # initialize our simulation kernel and instantiate the written rules
        tl.init()
        rule1 = Rule(10, eg_rule_action1, eg_rule_condition, None, None)
        r1.resource_manager.load(rule1)
        rule2 = Rule(10, eg_rule_action2, eg_rule_condition, None, None)
        r2.resource_manager.load(rule2)
    
        tick = time.time()
        tl.run()
        print("execution time %.2f sec" % (time.time() - tick))
    
    
    
        for info in r1.resource_manager.memory_manager:
            if info.entangle_time > 0:
                data.append(info.entangle_time / 1e12)
                
    # display our collected metrics
    #data.sort()
    plt.plot(data, range(1, len(data) + 1), marker="o")
    plt.xlabel("distance (km)")
    plt.ylabel("Entanglement gen rate, /1sec")
    plt.show()
    
    
    
    
    
    

In [35]:
interactive_plot = interact(test, cc_delay=(0.1, 10, 0.1), qc_atten=[1e-4, 2e-4, 3e-4], qc_dist_range=(10,50,1))
interactive_plot

interactive(children=(FloatSlider(value=5.0, description='cc_delay', max=10.0, min=0.1), Dropdown(description=…

<function __main__.test(cc_delay, qc_atten, qc_dist_range)>

In [ ]:
def test(sim_time, qc_atten):
    network_config = "topology.json"